In [21]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from nltk.corpus import gutenberg, stopwords
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [2]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    text = re.sub(r'CHAPTER \d+', '', text)
    text = re.sub("\\n\\n.*?\\n\\n", '', text)
  
    # Get rid of extra whitespace.
    text = ' '.join(text.split())

    
    return text

In [3]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

## Data Cleaning & Setup

In [4]:
# load documents
ball = gutenberg.raw('chesterton-ball.txt')
brown = gutenberg.raw('chesterton-brown.txt')
thursday = gutenberg.raw('chesterton-thursday.txt')

# clean texts
ball = text_cleaner(ball)
brown = text_cleaner(brown)
thursday = text_cleaner(thursday)

# run spacy and analyze the documents
nlp = spacy.load('en')
ball_doc = nlp(ball)
brown_doc = nlp(brown)
thursday_doc = nlp(thursday)

In [5]:
brown_sents = []
ball_sents = []
thursday_sents = []
for sentence in brown_doc.sents:
    brown_sents.append(sentence)
for sentence in ball_doc.sents:
    ball_sents.append(sentence)
for sentence in thursday_doc.sents:
    thursday_sents.append(sentence)

In [6]:
print('Length of\nBrown sentences: {}\nBall sentences: {}\nThursday sentences: {}'.format(len(brown_sents), len(ball_sents), len(thursday_sents)))

Length of
Brown sentences: 3534
Ball sentences: 4272
Thursday sentences: 3053


In [7]:
brown_sents = brown_sents[:2000]
ball_sents = ball_sents[:2000]
thursday_sents = thursday_sents[:2000]

In [8]:
# combine the documents into a dataframe with identifier
ball_df = [[sent, "Ball"] for sent in ball_sents]
brown_df = [[sent, "Brown"] for sent in brown_sents]
thursday_df = [[sent, "Thursday"] for sent in thursday_sents]

sentences = pd.DataFrame(ball_df + brown_df + thursday_df)
sentences.head()

,0,1
0,"(The, flying, ship, of, Professor, Lucifer, sa...",Ball
1,"(That, it, was, far, above, the, earth, was, n...",Ball
2,"(The, professor, had, himself, invented, the, ...",Ball
3,"(Every, sort, of, tool, or, apparatus, had, ,,...",Ball
4,"(For, the, world, of, science, and, evolution,...",Ball


In [9]:
len(sentences)

6000

## Bag of Words

In [16]:
# Utility function to create a list of the 750 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(750)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    print('Setting up the Dataframe...')
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 250 == 0:
            print("Processing row {}".format(i))
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
    
    return df

In [17]:
# Set up the bags.
ball_words = bag_of_words(ball_doc)
thursday_words = bag_of_words(thursday_doc)
brown_words = bag_of_words(brown_doc)

# Combine bags to create a set of unique words.
common_words = set(ball_words + thursday_words + brown_words)
len(common_words)

1212

In [18]:
# Create our data frame with features
word_counts = bow_features(sentences, common_words)
word_counts.head()

Setting up the Dataframe...
Processing row 0
Processing row 250
Processing row 500
Processing row 750
Processing row 1000
Processing row 1250
Processing row 1500
Processing row 1750
Processing row 2000
Processing row 2250
Processing row 2500
Processing row 2750
Processing row 3000
Processing row 3250
Processing row 3500
Processing row 3750
Processing row 4000
Processing row 4250
Processing row 4500
Processing row 4750
Processing row 5000
Processing row 5250
Processing row 5500
Processing row 5750


,be,close,angle,nerve,shin,remark,different,mankind,duke,private,...,symbol,blade,purple,expect,dare,crawl,capture,longer,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, flying, ship, of, Professor, Lucifer, sa...",Ball
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(That, it, was, far, above, the, earth, was, n...",Ball
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, professor, had, himself, invented, the, ...",Ball
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Every, sort, of, tool, or, apparatus, had, ,,...",Ball
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(For, the, world, of, science, and, evolution,...",Ball


In [22]:
# random forest fitting
rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9644444444444444

Test set score: 0.6091666666666666


In [23]:
# logistic regression fitting
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3600, 1212) (3600,)
Training set score: 0.8386111111111111

Test set score: 0.6616666666666666


In [24]:
# gradient boosting fitting
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.6822222222222222

Test set score: 0.61375


In [106]:
for word in word_counts:
    if word.sum() == 0:
        print('Word {} is 0'.format(word))

AttributeError: 'str' object has no attribute 'sum'

In [131]:
for word in common_words:
    if word_counts.loc[:, word].sum() == 0:
        print(word)
        continue

yacht
ebook
putnam
bert
renard
quayle
hose
boulnois
nigger
innkeeper
john
bandstand
claude
pooley
dalroy
madeleine
hutton
balloon
pendragon
ebooks
fanshaw
otto
cray
durand
kidd
castle
oman
gutenberg


## Tf-idf

In [147]:
# tf-idf function
def document_freq(data, sentences, common_words):
    
    # initialize df
    df = pd.DataFrame(columns = common_words)
    df.iloc[:, 0] = [0, 0, 0, 0, 0, 0]
    df.loc[:, common_words] = 0
    df.rename(index={0:'df', 1:'cf', 2:'idf', 3:'ball', 4:'brown', 5:'thursday'}, inplace=True)
    
    for word in common_words:
        # drop words that aren't existing; problem due to the way common words are set up
        if data.loc[:, word].sum() == 0:
            df.drop([word], axis=1, inplace=True)
            continue
        
        # find document frequency & collection frequency
        df.loc['df', word] = data[data[word] > 0][word].count()
        df.loc['cf', word] = data.loc[:, word].sum()
        
        # find idf
        df.loc['idf', word] = np.log2(len(sentences)/df.loc['df', word])
        
    
    return df

In [148]:
tf_idf = document_freq(word_counts, sentences, common_words)
tf_idf

,be,close,angle,nerve,shin,remark,different,mankind,duke,private,...,end,worth,symbol,blade,purple,expect,dare,crawl,capture,longer
df,243.000000,34.000000,8.000000,10.000000,13.000000,18.000000,15.000000,10.000000,23.000000,20.000000,...,85.000000,8.000000,13.000000,20.000000,15.000000,18.000000,11.000000,11.000000,13.000000,6.000000
cf,265.000000,34.000000,10.000000,10.000000,14.000000,19.000000,15.000000,11.000000,25.000000,20.000000,...,87.000000,9.000000,14.000000,20.000000,15.000000,19.000000,11.000000,12.000000,14.000000,7.000000
idf,4.625934,7.463284,9.550747,9.228819,8.850307,8.380822,8.643856,9.228819,8.027185,8.228819,...,6.141356,9.550747,8.850307,8.228819,8.643856,8.380822,9.091315,9.091315,8.850307,9.965784
ball,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
brown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
thursday,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [168]:
tf_idf.loc['ball', 'be']

0.0

In [170]:
# assign the idf value to the documents
for word in tf_idf.columns:
    if word in ball_words:
        tf_idf.loc['ball', word] = tf_idf.loc['idf', word]
    if word in brown_words:
        tf_idf.loc['brown', word] = tf_idf.loc['idf', word]
    if word in thursday_words:
        tf_idf.loc['thursday', word] = tf_idf.loc['idf', word]
tf_idf

,be,close,angle,nerve,shin,remark,different,mankind,duke,private,...,end,worth,symbol,blade,purple,expect,dare,crawl,capture,longer
df,243.000000,34.000000,8.000000,10.000000,13.000000,18.000000,15.000000,10.000000,23.000000,20.000000,...,85.000000,8.000000,13.000000,20.000000,15.000000,18.000000,11.000000,11.000000,13.000000,6.000000
cf,265.000000,34.000000,10.000000,10.000000,14.000000,19.000000,15.000000,11.000000,25.000000,20.000000,...,87.000000,9.000000,14.000000,20.000000,15.000000,19.000000,11.000000,12.000000,14.000000,7.000000
idf,4.625934,7.463284,9.550747,9.228819,8.850307,8.380822,8.643856,9.228819,8.027185,8.228819,...,6.141356,9.550747,8.850307,8.228819,8.643856,8.380822,9.091315,9.091315,8.850307,9.965784
ball,4.625934,7.463284,9.550747,0.000000,8.850307,8.380822,8.643856,0.000000,0.000000,0.000000,...,6.141356,0.000000,0.000000,8.228819,0.000000,8.380822,9.091315,0.000000,0.000000,9.965784
brown,4.625934,7.463284,9.550747,0.000000,0.000000,0.000000,8.643856,0.000000,8.027185,8.228819,...,6.141356,0.000000,0.000000,8.228819,8.643856,8.380822,0.000000,9.091315,8.850307,9.965784
thursday,4.625934,7.463284,0.000000,9.228819,8.850307,8.380822,0.000000,9.228819,0.000000,0.000000,...,6.141356,9.550747,8.850307,0.000000,0.000000,8.380822,0.000000,0.000000,0.000000,0.000000


In [171]:
tf_idf.T

,df,cf,idf,ball,brown,thursday
be,243.0,265.0,4.625934,4.625934,4.625934,4.625934
close,34.0,34.0,7.463284,7.463284,7.463284,7.463284
angle,8.0,10.0,9.550747,9.550747,9.550747,0.000000
nerve,10.0,10.0,9.228819,0.000000,0.000000,9.228819
shin,13.0,14.0,8.850307,8.850307,0.000000,8.850307
remark,18.0,19.0,8.380822,8.380822,0.000000,8.380822
different,15.0,15.0,8.643856,8.643856,8.643856,0.000000
mankind,10.0,11.0,9.228819,0.000000,0.000000,9.228819
duke,23.0,25.0,8.027185,0.000000,8.027185,0.000000
private,20.0,20.0,8.228819,0.000000,8.228819,0.000000


In [ ]:
for t in brown:
    print(t[0])

T
H
E
 
c
o
n
s
u
l
t
i
n
g
-
r
o
o
m
s
 
o
f
 
D
r
 
O
r
i
o
n
 
H
o
o
d
,
 
t
h
e
 
e
m
i
n
e
n
t
 
c
r
i
m
i
n
o
l
o
g
i
s
t
 
a
n
d
 
s
p
e
c
i
a
l
i
s
t
 
i
n
 
c
e
r
t
a
i
n
 
m
o
r
a
l
 
d
i
s
o
r
d
e
r
s
,
 
l
a
y
 
a
l
o
n
g
 
t
h
e
 
s
e
a
-
f
r
o
n
t
 
a
t
 
S
c
a
r
b
o
r
o
u
g
h
,
 
i
n
 
a
 
s
e
r
i
e
s
 
o
f
 
v
e
r
y
 
l
a
r
g
e
 
a
n
d
 
w
e
l
l
-
l
i
g
h
t
e
d
 
f
r
e
n
c
h
 
w
i
n
d
o
w
s
,
 
w
h
i
c
h
 
s
h
o
w
e
d
 
t
h
e
 
N
o
r
t
h
 
S
e
a
 
l
i
k
e
 
o
n
e
 
e
n
d
l
e
s
s
 
o
u
t
e
r
 
w
a
l
l
 
o
f
 
b
l
u
e
-
g
r
e
e
n
 
m
a
r
b
l
e
.
 
I
n
 
s
u
c
h
 
a
 
p
l
a
c
e
 
t
h
e
 
s
e
a
 
h
a
d
 
s
o
m
e
t
h
i
n
g
 
o
f
 
t
h
e
 
m
o
n
o
t
o
n
y
 
o
f
 
a
 
b
l
u
e
-
g
r
e
e
n
 
d
a
d
o
:
 
f
o
r
 
t
h
e
 
c
h
a
m
b
e
r
s
 
t
h
e
m
s
e
l
v
e
s
 
w
e
r
e
 
r
u
l
e
d
 
t
h
r
o
u
g
h
o
u
t
 
b
y
 
a
 
t
e
r
r
i
b
l
e
 
t
i
d
i
n
e
s
s
 
n
o
t
 
u
n
l
i
k
e
 
t
h
e
 
t
e
r
r
i
b
l
e
 
t
i
d
i
n
e
s
s
 
o
f
 
t
h
e
 
s
e
a
.
 
I
t
 
m
u
s
t
 
n
o
t
 
b
e
 
s
u
p
p
o
s
e


In [179]:
all_sents = []
for sentence brown_doc:
    sent = sentence[0]
    

spacy.tokens.doc.Doc

In [180]:
X_train, X_test = train_test_split(brown_sents+ball_sents+thursday_sents, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words=None, 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

#Applying the vectorizer
chesterton_tfidf = vectorizer.fit_transform(brown_doc+ball_doc+thursday_doc)
print("Number of features: %d" % chesterton_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(chesterton_tfidf, test_size=0.4, random_state=0)

#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of sentences
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per sentence
tfidf_bysent = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bysent[i][terms[j]] = X_train_tfidf_csr[i, j]
    
#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Original sentence:', X_train[5])
print('Tf_idf vector:', tfidf_bysent[5])

TypeError: unsupported operand type(s) for +: 'spacy.tokens.doc.Doc' and 'spacy.tokens.doc.Doc'

In [19]:
# # reduce feature set
# svd = TruncatedSVD(330)
# lsa = make_pipeline(svd, Normalizer(copy=False))

# # run SVD on training data then project on training data
# X_train_lsa = lsa.fit_transform(X_train_tf)

# variance_explained = svd.explained_variance_ratio_
# total_variance = variance_explained.sum()
# print("Percent variance captured by all components:", total_variance*100)

# #Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
# paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)
# for i in range(5):
#     print('Component {}:'.format(i))
#     print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

NameError: name 'X_train_tf' is not defined